In [ ]:
!pip install fpdf2 markdown2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.7/72.7 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.7/251.7 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.0/50.0 kB 3.6 MB/s eta 0:00:00


In [ ]:
"""
Cybersecurity Intelligence Platform - Multi-Agent System
A comprehensive threat intelligence system using free APIs and rule-based logic.
Generates both Markdown and PDF reports.
"""

import os
import json
import requests
import re
import time
from datetime import datetime, timedelta
from typing import List, Dict, Any
from dataclasses import dataclass, asdict
from fpdf import FPDF  # New import for PDF generation
import markdown2      # New import for HTML conversion

# ==================== Configuration ====================
@dataclass
class Config:
    """Configuration for the Cybersecurity Intelligence Platform"""

    # NVD API key is highly recommended for faster API access.
    # Get one here: https://nvd.nist.gov/developers/request-an-api-key
    # With no key: 5 requests per 30 seconds
    # With a key: 50 requests per 30 seconds
    NVD_API_KEY: str = ""  # Set your API key here, e.g., "your-api-key-here"

    # API Endpoints (Free Tier)
    NVD_API_URL: str = "https://services.nvd.nist.gov/rest/json/cves/2.0"

    # Local LLM Configuration (DISABLED FOR THIS VERSION)
    # Set to False to use the simple, rule-based logic for Agent 3.
    # This avoids any large downloads or GPU requirements.
    USE_LOCAL_LLM: bool = False
    OLLAMA_API_URL: str = "http://localhost:11434/api/chat"
    LLM_MODEL: str = "llama3:8b"  # Ignored when USE_LOCAL_LLM is False

    # Report Configuration
    REPORT_DIR: str = "./reports"
    DAYS_LOOKBACK: int =  10# How many days back to search for threats

# ==================== Data Models ====================
@dataclass
class Threat:
    """Represents a cybersecurity threat"""
    title: str
    description: str
    severity: str
    date: str
    source: str
    indicators: List[str]

@dataclass
class Vulnerability:
    """Represents a CVE vulnerability"""
    cve_id: str
    description: str
    severity: str
    cvss_score: float
    published_date: str
    affected_products: List[str]
    references: List[str]

@dataclass
class Mitigation:
    """Represents a mitigation strategy"""
    threat_type: str
    recommendations: List[str]
    priority: str
    implementation_steps: List[str]
    # NEW: This field links a specific mitigation to a specific threat or group
    source_title: str = "General"

# ==================== Agent 1: Threat Intelligence Analyst ====================
class ThreatIntelligenceAgent:
    """Fetches and analyzes real-time cybersecurity threats"""

    def __init__(self, config: Config):
        self.config = config
        self.threats: List[Threat] = []
        self.headers = {'User-Agent': 'CyberIntelPlatform/1.0'}
        if self.config.NVD_API_KEY:
            self.headers['apiKey'] = self.config.NVD_API_KEY

    def _api_wait(self):
        """Waits for the required NVD API rate limit interval"""
        # NVD's rate limit is 5 reqs/30s without a key, 50 reqs/30s with.
        # Waiting 6 seconds between requests is a safe bet for no key.
        # Waiting 1 second is fine with a key.
        wait_time = 1 if self.config.NVD_API_KEY else 6
        print(f"  ... Waiting {wait_time}s for NVD API rate limit ...")
        time.sleep(wait_time)

    def fetch_threat_feeds(self) -> List[Threat]:
        """Fetch threats from multiple free sources"""
        print("\n[Agent 1] 🔍 Threat Intelligence Analyst: Gathering threat data...")

        threats = []

        # Fetch from NVD Recent Vulnerabilities
        threats.extend(self._fetch_nvd_threats())

        # [TODO] Fetch from security news aggregators
        # threats.extend(self._fetch_security_news())
        # This function is currently a placeholder.

        # Simulate threat intelligence (fallback/augmentation)
        threats.extend(self._generate_sample_threats())

        self.threats = threats
        print(f"[Agent 1] ✅ Identified {len(threats)} active threats")
        return threats

    def _fetch_nvd_threats(self) -> List[Threat]:
        """Fetch recent critical vulnerabilities from NVD"""
        threats = []
        try:
            # Calculate date range
            end_date = datetime.now()
            start_date = end_date - timedelta(days=self.config.DAYS_LOOKBACK)

            params = {
                'pubStartDate': start_date.strftime('%Y-%m-%dT%H:%M:%S.000'),
                'pubEndDate': end_date.strftime('%Y-%m-%dT%H:%M:%S.000'),
                'resultsPerPage': 10,
                'cvssV3Severity': 'CRITICAL' # Only fetch critical threats
            }

            print("  → Fetching from NVD database...")
            response = requests.get(self.config.NVD_API_URL, params=params, headers=self.headers, timeout=10)

            if response.status_code == 200:
                data = response.json()
                vulnerabilities = data.get('vulnerabilities', [])

                for vuln in vulnerabilities[:5]:  # Limit to 5 most recent
                    cve_data = vuln.get('cve', {})
                    cve_id = cve_data.get('id', 'Unknown')

                    descriptions = cve_data.get('descriptions', [])
                    description = descriptions[0].get('value', 'No description') if descriptions else 'No description'

                    metrics = cve_data.get('metrics', {})
                    severity = 'UNKNOWN'
                    if 'cvssMetricV31' in metrics and metrics['cvssMetricV31']:
                        severity = metrics['cvssMetricV31'][0].get('cvssData', {}).get('baseSeverity', 'UNKNOWN')

                    threat = Threat(
                        title=f"Critical Vulnerability Exploit: {cve_id}",
                        description=description[:200] + "...",
                        severity=severity,
                        date=cve_data.get('published', 'Unknown'),
                        source="NVD",
                        indicators=[cve_id]
                    )
                    threats.append(threat)

                print(f"  ✓ Retrieved {len(threats)} threats from NVD")

            else:
                print(f"  ⚠ NVD fetch error: Status Code {response.status_code}")
                print(f"  Response: {response.text[:100]}...")

            # CRITICAL: Wait before next API call
            self._api_wait()

        except Exception as e:
            print(f"  ⚠ NVD fetch error: {e}")

        return threats

    def _fetch_security_news(self) -> List[Threat]:
        """[TODO] Fetch security news and incidents"""
        print("  → Skipping RSS feed fetch (function not implemented).")
        threats = []
        # This would integrate with RSS feeds (e.g., TheHackerNews)
        # using a library like 'feedparser'.
        # This is left as a future enhancement.
        return threats

    def _generate_sample_threats(self) -> List[Threat]:
        """Generate sample threats for demonstration (augments real data)"""
        print("  → Augmenting with sample threat intelligence...")
        return [
            Threat(
                title="Ransomware Campaign Targeting Healthcare Sector",
                description="New ransomware variant 'BlackCat 2.0' actively targeting healthcare organizations with double extortion tactics",
                severity="CRITICAL",
                date=(datetime.now() - timedelta(days=1)).strftime("%Y-%m-%d"),
                source="Simulated Threat Intel",
                indicators=["BlackCat", "ransomware", "healthcare"]
            ),
            Threat(
                title="Phishing Campaign Using Fake Microsoft 365 Login Pages",
                description="Widespread phishing campaign detected using lookalike Microsoft 365 domains to harvest credentials",
                severity="HIGH",
                date=(datetime.now() - timedelta(days=2)).strftime("%Y-%m-%d"),
                source="Simulated Threat Intel",
                indicators=["phishing", "credential-theft", "microsoft"]
            )
        ]

# ==================== Agent 2: Vulnerability Researcher ====================
class VulnerabilityResearchAgent:
    """Searches and analyzes CVE vulnerabilities"""

    def __init__(self, config: Config):
        self.config = config
        self.vulnerabilities: List[Vulnerability] = []
        self.headers = {'User-Agent': 'CyberIntelPlatform/1.0'}
        if self.config.NVD_API_KEY:
            self.headers['apiKey'] = self.config.NVD_API_KEY

    def _api_wait(self):
        """Waits for the required NVD API rate limit interval"""
        wait_time = 1 if self.config.NVD_API_KEY else 6
        print(f"  ... Waiting {wait_time}s for NVD API rate limit ...")
        time.sleep(wait_time)

    def research_vulnerabilities(self, threat_context: List[Threat]) -> List[Vulnerability]:
        """Research CVEs based on threat intelligence"""
        print("\n[Agent 2] 🔬 Vulnerability Researcher: Analyzing CVE database...")

        vulnerabilities = []

        # Fetch recent critical CVEs
        vulnerabilities.extend(self._fetch_recent_cves())

        # [TODO] Correlate with threat intelligence
        # vulnerabilities.extend(self._correlate_with_threats(threat_context))
        # This function is currently a placeholder.

        self.vulnerabilities = vulnerabilities
        print(f"[Agent 2] ✅ Analyzed {len(vulnerabilities)} vulnerabilities")
        return vulnerabilities

    def _fetch_recent_cves(self) -> List[Vulnerability]:
        """Fetch recent high-severity CVEs from NVD"""
        vulnerabilities = []

        try:
            end_date = datetime.now()
            start_date = end_date - timedelta(days=self.config.DAYS_LOOKBACK)

            params = {
                'pubStartDate': start_date.strftime('%Y-%m-%dT%H:%M:%S.000'),
                'pubEndDate': end_date.strftime('%Y-%m-%dT%H:%M:%S.000'),
                'resultsPerPage': 15,
                'cvssV3Severity': 'HIGH' # Get HIGH and CRITICAL
            }

            print("  → Querying CVE database for HIGH/CRITICAL CVEs...")
            response = requests.get(self.config.NVD_API_URL, params=params, headers=self.headers, timeout=10)

            if response.status_code == 200:
                data = response.json()
                cve_items = data.get('vulnerabilities', [])

                for item in cve_items[:10]:
                    cve_data = item.get('cve', {})
                    cve_id = cve_data.get('id', 'Unknown')

                    descriptions = cve_data.get('descriptions', [])
                    description = descriptions[0].get('value', 'No description') if descriptions else 'No description'

                    # Extract CVSS score and severity
                    metrics = cve_data.get('metrics', {})
                    cvss_score = 0.0
                    severity = 'UNKNOWN'

                    if 'cvssMetricV31' in metrics and metrics['cvssMetricV31']:
                        cvss_data = metrics['cvssMetricV31'][0].get('cvssData', {})
                        cvss_score = cvss_data.get('baseScore', 0.0)
                        severity = cvss_data.get('baseSeverity', 'UNKNOWN')

                    # Extract affected products
                    affected_products = []
                    configurations = cve_data.get('configurations', [])
                    for config in configurations:
                        nodes = config.get('nodes', [])
                        for node in nodes:
                            cpe_matches = node.get('cpeMatch', [])
                            for cpe in cpe_matches[:3]:  # Limit to 3 products
                                criteria = cpe.get('criteria', '')
                                if criteria:
                                    # Format as vendor:product
                                    parts = criteria.split(':')[3:5]
                                    if len(parts) == 2:
                                        affected_products.append(f"{parts[0]}:{parts[1]}")

                    # Extract references
                    references = []
                    refs = cve_data.get('references', [])
                    for ref in refs[:3]:
                        references.append(ref.get('url', ''))

                    vulnerability = Vulnerability(
                        cve_id=cve_id,
                        description=description,
                        severity=severity,
                        cvss_score=cvss_score,
                        published_date=cve_data.get('published', 'Unknown'),
                        affected_products=list(set(affected_products)), # Get unique products
                        references=references
                    )

                    vulnerabilities.append(vulnerability)

                print(f"  ✓ Found {len(vulnerabilities)} high-severity CVEs")

            else:
                print(f"  ⚠ NVD fetch error: Status Code {response.status_code}")
                print(f"  Response: {response.text[:100]}...")

            # CRITICAL: Wait before next API call
            self._api_wait()

        except Exception as e:
            print(f"  ⚠ CVE fetch error: {e}")

        return vulnerabilities

    def _correlate_with_threats(self, threats: List[Threat]) -> List[Vulnerability]:
        """[TODO] Correlate threats with known vulnerabilities"""
        print("  → Skipping threat correlation (function not implemented).")
        # Extract CVE IDs from threats
        cve_pattern = re.compile(r'CVE-\d{4}-\d{4,}')

        vulnerabilities = []
        for threat in threats:
            cve_matches = cve_pattern.findall(threat.description + " " + threat.title)
            for cve_id in cve_matches[:2]:  # Limit to 2 per threat
                # This would fetch detailed CVE info from NVD API
                # This is left as a future enhancement.
                pass

        return vulnerabilities

# ==================== Agent 3: Incident Response Advisor ====================
class IncidentResponseAgent:
    """Provides mitigation strategies and defensive measures"""

    def __init__(self, config: Config):
        self.config = config
        self.mitigations: List[Mitigation] = []

    def generate_response_strategies(self,
                                     threats: List[Threat],
                                     vulnerabilities: List[Vulnerability]) -> List[Mitigation]:
        """Generate mitigation strategies based on threats and vulnerabilities"""
        print("\n[Agent 3] 🛡️ Incident Response Advisor: Generating mitigation strategies...")

        mitigations = []

        if self.config.USE_LOCAL_LLM:
            # This block is skipped because USE_LOCAL_LLM is False
            print("  → Attempting to use Local LLM for analysis...")
            try:
                mitigations = self._analyze_with_llm(threats, vulnerabilities)
            except Exception as e:
                print(f"  ⚠ Local LLM call failed: {e}")
                print("  → Reverting to rule-based logic.")
                mitigations = self._analyze_with_rules(threats, vulnerabilities)
        else:
            # This block is EXECUTED
            print("  → Using rule-based logic (USE_LOCAL_LLM=False).")
            mitigations = self._analyze_with_rules(threats, vulnerabilities)

        # Add general security recommendations
        mitigations.extend(self._generate_general_recommendations())

        self.mitigations = mitigations
        print(f"[Agent 3] ✅ Generated {len(mitigations)} mitigation strategies")
        return mitigations

    def _analyze_with_llm(self, threats: List[Threat], vulnerabilities: List[Vulnerability]) -> List[Mitigation]:
        """
        [SKIPPED] Generate mitigations using a local LLM (e.g., Ollama).
        This function is not called when USE_LOCAL_LLM is False.
        """

        print("  → [SKIPPED] LLM analysis is disabled.")
        return []

        # --- The code below is NOT executed in this version ---

        # Convert data models to simple dicts for JSON serialization
        threat_data = [asdict(t) for t in threats]
        vuln_data = [asdict(v) for v in vulnerabilities]

        # Create the prompt for the LLM
        prompt = f"""
        You are an expert Cybersecurity Incident Response Advisor.
        Analyze the following JSON data of active threats and new vulnerabilities.

        Active Threats:
        {json.dumps(threat_data, indent=2)}

        New Vulnerabilities:
        {json.dumps(vuln_data, indent=2)}

        Based *only* on this data, generate a concise list of mitigation strategies.
        For each mitigation, provide:
        1. threat_type (e.g., "Ransomware", "Critical Vulnerability")
        2. recommendations (a list of 2-3 brief, high-level actions)
        3. priority ("CRITICAL", "HIGH", or "MEDIUM")
        4. implementation_steps (a list of 2-3 specific, actionable steps)
        5. source_title (the 'title' of the threat this mitigation is for)

        Respond *only* with a valid JSON list `[...]` containing the mitigation objects.
        Do not include any other text, pre-amble, or explanations.
        """

        # --- Call Ollama API ---
        payload = {
            "model": self.config.LLM_MODEL,
            "messages": [
                {"role": "system", "content": "You are a cybersecurity expert responding in JSON."},
                {"role": "user", "content": prompt}
            ],
            "format": "json", # Request JSON format
            "stream": False
        }

        print(f"  → Sending {len(prompt)} chars to Ollama model {self.config.LLM_MODEL}...")
        response = requests.post(self.config.OLLAMA_API_URL, json=payload, timeout=120)
        response.raise_for_status() # Raise an error for bad responses

        response_data = response.json()
        llm_json_output = response_data.get('message', {}).get('content', '[]')

        # --- Parse LLM Response ---
        llm_mitigations = json.loads(llm_json_output)

        mitigations = []
        for m in llm_mitigations:
            mitigations.append(Mitigation(
                threat_type=m.get('threat_type', 'Unknown Threat'),
                recommendations=m.get('recommendations', []),
                priority=m.get('priority', 'MEDIUM'),
                implementation_steps=m.get('implementation_steps', []),
                source_title=m.get('source_title', 'General')
            ))

        print(f"  ✓ Successfully generated {len(mitigations)} mitigations from LLM.")
        return mitigations


    def _analyze_with_rules(self, threats: List[Threat], vulnerabilities: List[Vulnerability]) -> List[Mitigation]:
        """Generate mitigations for identified threats using simple rules"""
        mitigations = []

        # NEW: Create specific mitigations for each threat
        for threat in threats:
            mitigation = None
            if 'ransomware' in threat.title.lower():
                mitigation = Mitigation(
                    threat_type="Ransomware Attack",
                    recommendations=[
                        "Implement robust backup and recovery procedures (3-2-1 rule)",
                        "Deploy endpoint detection and response (EDR) solutions",
                    ],
                    priority="CRITICAL",
                    implementation_steps=[
                        "Verify all critical data backups are offline and immutable",
                        "Configure EDR to block common ransomware behaviors",
                    ],
                    source_title=threat.title  # <-- NEW: Linking action to threat
                )
            elif 'phishing' in threat.title.lower():
                mitigation = Mitigation(
                    threat_type="Phishing Campaign",
                    recommendations=[
                        "Deploy multi-factor authentication (MFA) across all external systems",
                        "Implement email authentication (SPF, DKIM, DMARC)",
                    ],
                    priority="HIGH",
                    implementation_steps=[
                        "Enforce MFA for all user accounts immediately",
                        "Configure DMARC policy to reject or quarantine",
                    ],
                    source_title=threat.title  # <-- NEW: Linking action to threat
                )
            elif 'vulnerability' in threat.title.lower():
                mitigation = Mitigation(
                    threat_type="Zero-Day or Critical Exploitation",
                    recommendations=[
                        "Apply vendor-supplied patches or workarounds immediately",
                        "Restrict network access to vulnerable systems",
                    ],
                    priority="CRITICAL",
                    implementation_steps=[
                        "Identify all affected systems using vulnerability scanner",
                        "Apply virtual patching if official patch is unavailable",
                        f"Hunt for IOCs related to {threat.indicators[0]}"
                    ],
                    source_title=threat.title  # <-- NEW: Linking action to threat
                )

            if mitigation:
                mitigations.append(mitigation)

        # NEW: Create grouped mitigations for vulnerabilities
        critical_cves = [v for v in vulnerabilities if v.severity == 'CRITICAL']
        high_cves = [v for v in vulnerabilities if v.severity == 'HIGH']

        if critical_cves:
            mitigation = Mitigation(
                threat_type="Critical Vulnerability Patching",
                recommendations=[
                    f"Patch all {len(critical_cves)} identified CRITICAL vulnerabilities within 72 hours",
                    "Prioritize internet-facing systems",
                ],
                priority="CRITICAL",
                implementation_steps=[
                    "Identify all affected systems using vulnerability scanner",
                    f"Patches to apply include: {', '.join([c.cve_id for c in critical_cves[:2]])}...",
                ],
                source_title="Critical Vulnerability Group"  # <-- NEW: Linking action to a group
            )
            mitigations.append(mitigation)

        if high_cves:
            mitigation = Mitigation(
                threat_type="High-Severity Vulnerability Patching",
                recommendations=[
                    f"Address all {len(high_cves)} identified HIGH vulnerabilities within 14 days",
                    "Follow standard patch management procedures",
                ],
                priority="HIGH",
                implementation_steps=[
                    "Schedule patches in the next available maintenance window",
                    "Verify remediation with a follow-up vulnerability scan",
                ],
                source_title="High Vulnerability Group"  # <-- NEW: Linking action to a group
            )
            mitigations.append(mitigation)

        return mitigations

    def _generate_general_recommendations(self) -> List[Mitigation]:
        """Generate general security hardening recommendations"""
        return [
            Mitigation(
                threat_type="General Security Posture Enhancement",
                recommendations=[
                    "Conduct regular security assessments and penetration testing",
                    "Maintain a comprehensive asset inventory",
                    "Enforce the principle of least privilege"
                ],
                priority="MEDIUM",
                implementation_steps=[
                    "Schedule quarterly internal/external vulnerability scans",
                    "Deploy a CMDB or asset management solution",
                    "Review and revoke unnecessary user permissions monthly"
                ],
                source_title="General" # <-- This is a general mitigation
            )
        ]

# ==================== Agent 4: Report Writer ====================
class ReportWriterAgent:
    """Compiles comprehensive intelligence reports in Markdown and PDF"""

    def __init__(self, config: Config):
        self.config = config
        os.makedirs(config.REPORT_DIR, exist_ok=True)

    def generate_report(self,
                        threats: List[Threat],
                        vulnerabilities: List[Vulnerability],
                        mitigations: List[Mitigation]) -> str:
        """Generate comprehensive cybersecurity intelligence report"""
        print("\n[Agent 4] 📝 Report Writer: Compiling intelligence report...")

        report_date = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        report_filename_base = f"cyber_intel_report_{datetime.now().strftime('%Y%m%d_%H%M%S')}"
        md_path = os.path.join(self.config.REPORT_DIR, f"{report_filename_base}.md")
        pdf_path = os.path.join(self.config.REPORT_DIR, f"{report_filename_base}.pdf")

        # Build report sections
        report_content_md = self._build_markdown_report(report_date, threats, vulnerabilities, mitigations)

        # Save Markdown report
        try:
            with open(md_path, 'w', encoding='utf-8') as f:
                f.write(report_content_md)
            print(f"[Agent 4] ✅ Markdown Report saved to: {md_path}")
        except Exception as e:
            print(f"[Agent 4] ⚠ Error saving Markdown: {e}")

        # NEW: Generate PDF report
        try:
            self._generate_pdf_report(report_content_md, pdf_path)
            print(f"[Agent 4] ✅ PDF Report saved to: {pdf_path}")
        except Exception as e:
            print(f"[Agent 4] ⚠ Error generating PDF: {e}")
            print("     (This can happen if 'fpdf2' or 'markdown2' are not installed)")

        return report_content_md

    def _generate_pdf_report(self, markdown_content: str, pdf_path: str):
        """Converts Markdown content to a PDF file"""

        # 1. Clean the text: Remove emojis and special chars that FPDF can't handle
        # This regex removes most non-ASCII chars including emojis
        cleaned_md = re.sub(r'[^\x00-\x7F]+', '', markdown_content)

        # 2. Convert Markdown to HTML
        # Using 'fenced-code-blocks' and 'tables' extras for better formatting
        html_content = markdown2.markdown(cleaned_md, extras=["fenced-code-blocks", "tables"])

        # 3. Create PDF
        pdf = FPDF()
        pdf.add_page()
        pdf.set_font("Helvetica", size=10)

        # 4. Write the HTML content to the PDF
        # We must add a cell_height (h=5) or it won't render lists well
        pdf.write_html(html_content, 5)

        # 5. Save the PDF
        pdf.output(pdf_path)


    def _build_markdown_report(self,
                               report_date: str,
                               threats: List[Threat],
                               vulnerabilities: List[Vulnerability],
                               mitigations: List[Mitigation]) -> str:
        """Build comprehensive markdown report"""

        report = f"""# Cybersecurity Intelligence Report
**Generated:** {report_date}
**Analysis Period:** Last {self.config.DAYS_LOOKBACK} days
**Report Classification:** CONFIDENTIAL

---

## Executive Summary

This report provides a comprehensive analysis of the current cybersecurity threat landscape. The primary focus is a direct "Threat-to-Action" model, outlining immediate responses to active threats.

### Threat Level Assessment
"""

        critical_count = sum(1 for t in threats if t.severity == 'CRITICAL')
        critical_vulns = sum(1 for v in vulnerabilities if v.severity == 'CRITICAL')

        if critical_count > 2 or critical_vulns > 3:
            threat_level = "🔴 **CRITICAL** - Immediate action required"
        elif critical_count > 0 or critical_vulns > 0:
            threat_level = "🟠 **HIGH** - Urgent attention needed"
        else:
            threat_level = "🟡 **ELEVATED** - Standard monitoring required"
        report += f"\n**Current Threat Level:** {threat_level}\n\n"

        # --- NEW SECTION 1: Threat Analysis & Response ---
        report += "---\n\n## 1. Active Threat Analysis & Response\n\n"
        report += "This section links active threats directly to their recommended mitigation strategies.\n\n"

        threat_mitigations = [m for m in mitigations if m.source_title in [t.title for t in threats]]

        if not threats:
             report += "No active threats identified in this analysis period.\n\n"

        for i, threat in enumerate(threats, 1):
            severity_emoji = {'CRITICAL': '🔴', 'HIGH': '🟠'}.get(threat.severity, '⚪')
            report += f"### 1.{i} Threat: {threat.title}\n\n"
            report += f"**Severity:** {severity_emoji} {threat.severity}  \n"
            report += f"**Date:** {threat.date}  \n"
            report += f"**Source:** {threat.source}  \n"
            report += f"**Description:** {threat.description}\n\n"

            # Find the matching action
            action = next((m for m in threat_mitigations if m.source_title == threat.title), None)

            if action:
                report += f"**🔵 RECOMMENDED ACTION:**\n\n"
                report += f"**Priority:** {action.priority}\n"
                report += "**Recommendations:**\n"
                for rec in action.recommendations:
                    report += f"- {rec}\n"
                report += "\n**Implementation Steps:**\n"
                for step in action.implementation_steps:
                    report += f"- {step}\n"
            else:
                report += "**RECOMMENDED ACTION:**\n\n"
                report += "- No specific mitigation defined. Follow general guidance.\n"
            report += "\n---\n"

        # --- NEW SECTION 2: Vulnerability Analysis & Response ---
        report += "\n\n## 2. Vulnerability Analysis & Response\n\n"
        report += "This section lists high-impact vulnerabilities and their grouped mitigation plans.\n\n"

        if not vulnerabilities:
            report += "No high-severity vulnerabilities identified in this analysis period.\n\n"
        else:
            # Find grouped vulnerability mitigations
            crit_action = next((m for m in mitigations if m.source_title == "Critical Vulnerability Group"), None)
            high_action = next((m for m in mitigations if m.source_title == "High Vulnerability Group"), None)

            if crit_action:
                report += f"### 2.1 Critical Vulnerability Response ({crit_action.priority})\n"
                report += "**Recommendations:**\n"
                for rec in crit_action.recommendations:
                    report += f"- {rec}\n"
                report += "\n**Implementation Steps:**\n"
                for step in crit_action.implementation_steps:
                    report += f"- {step}\n"
                report += "\n"

            if high_action:
                report += f"### 2.2 High Vulnerability Response ({high_action.priority})\n"
                report += "**Recommendations:**\n"
                for rec in high_action.recommendations:
                    report += f"- {rec}\n"
                report += "\n**Implementation Steps:**\n"
                for step in high_action.implementation_steps:
                    report += f"- {step}\n"
                report += "\n"

            report += "#### **Detailed Vulnerability List:**\n\n"
            for i, vuln in enumerate(vulnerabilities, 1):
                severity_emoji = {'CRITICAL': '🔴', 'HIGH': '🟠'}.get(vuln.severity, '⚪')
                report += f"**{i}. {vuln.cve_id}** (CVSS: {vuln.cvss_score} - {vuln.severity} {severity_emoji})\n"
                report += f"   - **Description:** {vuln.description[:150]}...\n"
                if vuln.affected_products:
                    report += f"   - **Products:** `{', '.join(vuln.affected_products[:2])}`\n"
                report += "\n"

        # --- NEW SECTION 3: General Recommendations ---
        report += "\n\n## 3. General Security Posture Recommendations\n\n"
        general_actions = [m for m in mitigations if m.source_title == "General"]
        if general_actions:
            for action in general_actions:
                report += f"**Priority:** {action.priority}\n"
                report += "**Recommendations:**\n"
                for rec in action.recommendations:
                    report += f"- {rec}\n"
                report += "\n**Implementation Steps:**\n"
                for step in action.implementation_steps:
                    report += f"- {step}\n"
        else:
            report += "No general recommendations provided.\n"

        # --- Section 4: Metrics ---
        report += "\n---\n\n## 4. Security Metrics\n\n"
        report += f"| Metric | Value |\n"
        report += f"|--------|-------|\n"
        report += f"| Total Threats Identified | {len(threats)} |\n"
        report += f"| Critical Threats | {critical_count} |\n"
        report += f"| Total Vulnerabilities | {len(vulnerabilities)} |\n"
        report += f"| Critical CVEs | {critical_vulns} |\n"

        # --- Section 5: Footer ---
        report += "\n---\n\n## 5. Report Generation Details\n\n"
        report += f"**System:** Cybersecurity Intelligence Platform v1.0  \n"
        report += f"**Analysis Engine:** Rule-Based System (Multi-Agent AI Disabled)  \n"
        report += f"**Data Sources:** NVD, Simulated Feeds  \n"

        return report

# ==================== Orchestrator ====================
class CyberIntelligencePlatform:
    """Main orchestrator for the multi-agent system"""

    def __init__(self):
        self.config = Config()
        self.threat_agent = ThreatIntelligenceAgent(self.config)
        self.vuln_agent = VulnerabilityResearchAgent(self.config)
        self.response_agent = IncidentResponseAgent(self.config)
        self.report_agent = ReportWriterAgent(self.config)

    def run_analysis(self):
        """Execute the full intelligence analysis pipeline"""
        print("="*70)
        print("🔐 CYBERSECURITY INTELLIGENCE PLATFORM")
        print("="*70)
        print(f"Analysis Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
        print(f"Lookback Period: {self.config.DAYS_LOOKBACK} days")
        if self.config.NVD_API_KEY:
            print("NVD API Key: Loaded")
        else:
            print("NVD API Key: ⚠️ None (using public, rate-limited access)")
        if self.config.USE_LOCAL_LLM:
             print("AI Engine: 🧠 Local LLM (Active)")
        else:
             print("AI Engine: ⚙️ Rule-Based System (Local LLM Disabled)")
        print("="*70)

        # Stage 1: Threat Intelligence Collection
        threats = self.threat_agent.fetch_threat_feeds()

        # Stage 2: Vulnerability Research
        vulnerabilities = self.vuln_agent.research_vulnerabilities(threats)

        # Stage 3: Incident Response Planning
        mitigations = self.response_agent.generate_response_strategies(threats, vulnerabilities)

        # Stage 4: Report Generation
        report_md = self.report_agent.generate_report(threats, vulnerabilities, mitigations)

        # Summary
        print("\n" + "="*70)
        print("📊 ANALYSIS COMPLETE")
        print("="*70)
        print(f"✓ Threats Analyzed: {len(threats)}")
        print(f"✓ Vulnerabilities Identified: {len(vulnerabilities)}")
        print(f"✓ Mitigations Recommended: {len(mitigations)}")
        print(f"✓ Reports Generated in: {self.config.REPORT_DIR}")
        print("="*70)

        return report_md

# ==================== Main Entry Point ====================
def main():
    """Main entry point for the application"""
    try:
        platform = CyberIntelligencePlatform()
        report = platform.run_analysis()

        # Display report preview
        print("\n" + "="*70)
        print("📄 MARKDOWN REPORT PREVIEW (First 500 characters)")
        print("="*70)
        print(report[:500] + "...")
        print("="*70)

    except KeyboardInterrupt:
        print("\n\n⚠️ Analysis interrupted by user")
    except requests.exceptions.ConnectionError as e:
        print(f"\n\n❌ CONNECTION ERROR: Could not connect to API.")
        print(f"   Please check your internet connection and API URLs.")
    except Exception as e:
        print(f"\n\n❌ UNEXPECTED ERROR during analysis: {e}")
        import traceback
        traceback.print_exc()

if __name__ == "__main__":
    main()



🔐 CYBERSECURITY INTELLIGENCE PLATFORM
Analysis Date: 2025-10-23 04:36:58
Lookback Period: 10 days
NVD API Key: Loaded
AI Engine: ⚙️ Rule-Based System (Local LLM Disabled)

[Agent 1] 🔍 Threat Intelligence Analyst: Gathering threat data...
  → Fetching from NVD database...
  ✓ Retrieved 5 threats from NVD
  ... Waiting 1s for NVD API rate limit ...
  → Augmenting with sample threat intelligence...
[Agent 1] ✅ Identified 7 active threats

[Agent 2] 🔬 Vulnerability Researcher: Analyzing CVE database...
  → Querying CVE database for HIGH/CRITICAL CVEs...
  ✓ Found 10 high-severity CVEs
  ... Waiting 1s for NVD API rate limit ...
[Agent 2] ✅ Analyzed 10 vulnerabilities

[Agent 3] 🛡️ Incident Response Advisor: Generating mitigation strategies...
  → Using rule-based logic (USE_LOCAL_LLM=False).
[Agent 3] ✅ Generated 9 mitigation strategies

[Agent 4] 📝 Report Writer: Compiling intelligence report...
[Agent 4] ✅ Markdown Report saved to: ./reports/cyber_intel_report_20251023_043702.md
[Agent 4